# Import libraries

In [449]:
import pandas as pd
import numpy as np
from  sklearn.preprocessing import StandardScaler,OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,classification_report
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
import pickle
import os

# Import Data

In [450]:
df = pd.read_csv("../Dataset/health_lifestyle_classification.csv")
df.columns

Index(['survey_code', 'age', 'gender', 'height', 'weight', 'bmi',
       'bmi_estimated', 'bmi_scaled', 'bmi_corrected', 'waist_size',
       'blood_pressure', 'heart_rate', 'cholesterol', 'glucose', 'insulin',
       'sleep_hours', 'sleep_quality', 'work_hours', 'physical_activity',
       'daily_steps', 'calorie_intake', 'sugar_intake', 'alcohol_consumption',
       'smoking_level', 'water_intake', 'screen_time', 'stress_level',
       'mental_health_score', 'mental_health_support', 'education_level',
       'job_type', 'occupation', 'income', 'diet_type', 'exercise_type',
       'device_usage', 'healthcare_access', 'insurance', 'sunlight_exposure',
       'meals_per_day', 'caffeine_intake', 'family_history', 'pet_owner',
       'electrolyte_level', 'gene_marker_flag', 'environmental_risk_score',
       'daily_supplement_dosage', 'target'],
      dtype='object')

# Train test split

In [451]:
X = df.drop('target',axis=1)
y = df['target']
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=11,test_size=.25)

# Preprocessing

## Remove nulls / duplicates

In [452]:
def remove_null_duplicates(df):
    df = df.dropna()
    df = df.drop_duplicates()
    return df 

X_train = remove_null_duplicates(X_train)
X_test = remove_null_duplicates(X_test)

# match y index with corresponding x
train_index = X_train.index
test_index = X_test.index
y_train = y_train.loc[train_index]
y_test = y_test.loc[test_index]


## 5 point summary

In [453]:
summary = df.describe()
summary.T

,count,mean,std,min,25%,50%,75%,max
survey_code,100000.0,50000.500000,28867.657797,1.000000,25000.750000,50000.500000,75000.250000,100000.000000
age,100000.0,48.525990,17.886768,18.000000,33.000000,48.000000,64.000000,79.000000
height,100000.0,170.023707,9.982798,140.000000,163.306615,170.016778,176.728920,210.000000
weight,100000.0,70.064862,14.693667,40.000000,59.856938,69.924141,80.027418,139.250894
bmi,100000.0,24.493876,5.951069,9.988495,20.271405,24.156734,28.258696,59.234792
bmi_estimated,100000.0,24.493876,5.951069,9.988495,20.271405,24.156734,28.258696,59.234792
bmi_scaled,100000.0,73.481627,17.853206,29.965484,60.814215,72.470201,84.776088,177.704377
bmi_corrected,100000.0,24.494140,5.954184,9.893845,20.271059,24.151699,28.247648,59.142646
waist_size,100000.0,84.933043,12.040314,34.093185,76.795185,84.957139,93.018713,133.153631
blood_pressure,92331.0,119.980149,15.015503,59.128168,109.812060,119.951794,130.120621,184.439195


## Correlation matrics

In [454]:
num_data = X_train.select_dtypes(include=[int,float])
num_data.corr()

,survey_code,age,height,weight,bmi,bmi_estimated,bmi_scaled,bmi_corrected,waist_size,blood_pressure,...,water_intake,screen_time,stress_level,mental_health_score,income,meals_per_day,electrolyte_level,gene_marker_flag,environmental_risk_score,daily_supplement_dosage
survey_code,1.000000,0.007951,-0.011659,-0.004390,0.001207,0.001207,0.001207,0.001253,-0.012145,0.016916,...,0.007046,0.018800,0.004465,-0.005946,-0.009655,-0.013951,NaN,NaN,NaN,0.032310
age,0.007951,1.000000,-0.012157,0.001663,0.005697,0.005697,0.005697,0.005546,0.005334,-0.008125,...,0.004892,-0.008155,0.012583,0.005557,0.001858,-0.008385,NaN,NaN,NaN,-0.010159
height,-0.011659,-0.012157,1.000000,0.010980,-0.479509,-0.479509,-0.479509,-0.479098,-0.000020,0.007407,...,-0.002107,-0.001482,-0.003479,-0.002995,0.012179,-0.005167,NaN,NaN,NaN,-0.005716
weight,-0.004390,0.001663,0.010980,1.000000,0.863839,0.863839,0.863839,0.863420,0.004689,0.001283,...,0.004591,0.017155,0.007664,-0.020338,-0.008517,-0.016276,NaN,NaN,NaN,0.000392
bmi,0.001207,0.005697,-0.479509,0.863839,1.000000,1.000000,1.000000,0.999436,0.002160,-0.003283,...,0.005545,0.015754,0.006082,-0.018230,-0.013101,-0.012922,NaN,NaN,NaN,0.001483
bmi_estimated,0.001207,0.005697,-0.479509,0.863839,1.000000,1.000000,1.000000,0.999436,0.002160,-0.003283,...,0.005545,0.015754,0.006082,-0.018230,-0.013101,-0.012922,NaN,NaN,NaN,0.001483
bmi_scaled,0.001207,0.005697,-0.479509,0.863839,1.000000,1.000000,1.000000,0.999436,0.002160,-0.003283,...,0.005545,0.015754,0.006082,-0.018230,-0.013101,-0.012922,NaN,NaN,NaN,0.001483
bmi_corrected,0.001253,0.005546,-0.479098,0.863420,0.999436,0.999436,0.999436,1.000000,0.001475,-0.003064,...,0.005945,0.015683,0.006214,-0.017675,-0.013328,-0.012103,NaN,NaN,NaN,0.001764
waist_size,-0.012145,0.005334,-0.000020,0.004689,0.002160,0.002160,0.002160,0.001475,1.000000,0.004232,...,-0.004483,0.000885,-0.003523,-0.009865,0.010369,0.001624,NaN,NaN,NaN,-0.004159
blood_pressure,0.016916,-0.008125,0.007407,0.001283,-0.003283,-0.003283,-0.003283,-0.003064,0.004232,1.000000,...,-0.016879,0.014496,0.010216,0.002732,-0.004268,-0.002411,NaN,NaN,NaN,0.000864


## Rmove unnecessory columns 

In [455]:
def rem_columns(df):
    df = df.drop(['electrolyte_level', 'gene_marker_flag', 'environmental_risk_score'],axis=1)
    return df

X_train = rem_columns(X_train)
X_test = rem_columns(X_test)

## Encoding

In [456]:
encoding_mapper = {}
encoder = OrdinalEncoder()
X_train[X_train.select_dtypes(include='object').columns] = encoder.fit_transform(X_train.select_dtypes(include='object'))
X_test[X_test.select_dtypes(include='object').columns] = encoder.transform(X_test.select_dtypes(include='object'))


with open ('../pickles/enocoder.pkl','wb') as file:
    pickle.dump(encoder,file)


## Scaling

In [457]:
scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train),columns = X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test),columns = X_test.columns)

# Modeling

## Logistic Regression

In [ ]:
# Model
logreg = DecisionTreeClassifier(random_state=11)
# Training
logreg.fit(X_train,y_train)
# Prediction
y_pred_logreg = logreg.predict(X_test)
# Metrics
classification_report = classification_report(y_test,y_pred_logreg)
print(classification_report)

              precision    recall  f1-score   support

    diseased       0.29      0.31      0.30      1071
     healthy       0.70      0.69      0.70      2534

    accuracy                           0.58      3605
   macro avg       0.50      0.50      0.50      3605
weighted avg       0.58      0.58      0.58      3605



In [459]:
set((y_pred_logreg))

{'diseased', 'healthy'}

In [460]:
set(y_train)

{'diseased', 'healthy'}